In [ ]:
import pandas as pd
import numpy as np
from river import drift
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.simplefilter(action='ignore')

In [153]:
df = pd.read_csv('C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/gpx_processed_data.csv')
df

,activity_date,activity_name,activity_type,total_distance (km),total_time (min),pace (min/km),velocity (km/h),days_between_activities
0,2020-07-09,Corrida matinal,Atividade,6.03,39.0,6.47,9.27,0
1,2020-07-10,Pedalada matinal,Atividade,25.35,140.0,5.52,10.87,0
2,2020-07-11,NaN,Dia sem atividade,NaN,NaN,NaN,NaN,1
3,2020-07-12,NaN,Dia sem atividade,NaN,NaN,NaN,NaN,2
4,2020-07-13,Tornozelo doendo,Atividade,7.57,55.0,7.27,8.25,0
...,...,...,...,...,...,...,...,...
704,2022-06-13,Corrida matinal,Atividade,11.34,73.0,6.44,9.32,0
705,2022-06-14,NaN,Dia sem atividade,NaN,NaN,NaN,NaN,1
706,2022-06-15,NaN,Dia sem atividade,NaN,NaN,NaN,NaN,2
707,2022-06-16,NaN,Dia sem atividade,NaN,NaN,NaN,NaN,3


In [ ]:
df_athlete_activities = df[['activity_date', 'activity_type','days_between_activities']]

df_athlete_activities['activity_type'] = (df_athlete_activities['activity_type'] == 'Atividade') * 1
df_athlete_activities

In [ ]:
activity_MM = df_athlete_activities['activity_type'].rolling(14).sum()

df_athlete_activities['activity_moving_sum'] = activity_MM
df_athlete_activities

In [ ]:
examples_dict = []

active_period = False

for index, row in df_athlete_activities.iterrows():
    if (row['activity_moving_sum'] >= 6) & (active_period == False):
        active_period = True
        active_period_dict = {'start': row['activity_date']}

    elif (row['activity_moving_sum'] == 0) & (active_period == True):
        active_period_dict['end'] = row['activity_date']
        active_period = False    
        examples_dict.append(active_period_dict)

examples_dict

In [ ]:
fig, ax = plt.subplots(figsize =(26, 7))
sns.lineplot(x = 'activity_date', y= 'days_between_activities', data= df_athlete_activities)

for example in examples_dict:
    plt.axvline(x = example['start'], color = 'g', linestyle = '--', label = 'start active period')
    plt.axvline(x = example['end'], color = 'r', linestyle = '--', label = 'end active period')  

plt.xticks([])
#plt.title('Data Drift Detection')
plt.legend()
plt.show()

In [ ]:
drift_indexes = []

adwin = drift.ADWIN()

for index, row in df_athlete_activities.iterrows():
    adwin.update(row['days_between_activities'])
    if adwin.drift_detected:
        print(f"Change detected at index: {index}, date {row['activity_date']}, input value: {row['days_between_activities']}")
        drift_indexes.append(index)
        drift_index = index

In [ ]:
fig, ax = plt.subplots(figsize =(26, 7))
sns.lineplot(x = 'activity_date', y= 'days_between_activities', data= df_athlete_activities)


plt.axvline(x = drift_index, color = 'r', linestyle = '--')

plt.xticks([])
plt.title('Data Drift Detection')
plt.show()